In [1]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Feature_Importance_Plot_Random_Forest_Regressor.ipynb

In [2]:
def Decision_Tree_Regression(X_train, X_test, y_train, y_test, filtered_data):
    # Function to be optimized by Bayesian Optimization
    def dt_regressor_cv(max_depth, min_samples_split, min_samples_leaf):
        # Ensure parameters are in the correct format
        max_depth = int(max_depth)
        min_samples_split = int(min_samples_split)
        min_samples_leaf = int(min_samples_leaf)

        # Define and fit the model
        model = DecisionTreeRegressor(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=0
        )
        model.fit(X_train, y_train)

        # Predict and calculate R2 score
        predict_test = model.predict(X_test)
        r2_score_test = r2_score(y_test, predict_test)
        return r2_score_test

    # Setting ranges for hyperparameters
    pbounds = {
        'max_depth': (1, 32),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 10)
    }

    # Bayesian Optimization
    optimizer = BayesianOptimization(
        f=dt_regressor_cv,
        pbounds=pbounds,
        random_state=0
    )
    optimizer.maximize(init_points=5, n_iter=15)

    # Optimal parameters
    params = optimizer.max['params']
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])
    
    print("Optimal Parameters:", params)

    # Create and train model with optimal parameters
    best_model = DecisionTreeRegressor(
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        random_state=0
    )
    best_model.fit(X_train, y_train)

    # Predictions
    predict_train = best_model.predict(X_train)
    predict_test = best_model.predict(X_test)

    # DataFrame Creation
    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    # Concatenating and Sorting
    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()

    # R2 Score
    r2_score_train = round(r2_score(y_train, predict_train), 2)
    r2_score_test = round(r2_score(y_test, predict_test), 2)
    print('R2 Score Train:', r2_score_train)
    print('R2 Score Test:', r2_score_test)

    return best_model, params, r2_score_test
